<a href="https://colab.research.google.com/github/JeCase/TUBES_VISDAT_SAHAM/blob/main/TUBES_Visualisasi_Data_Saham.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INTERAKTIF VISUALISASI PERGERAKAN NILAI SAHAM PT MEDIA NUSANTARA CITRA MENGGUNAKAN BOKEH**

---

Disusun Oleh:
- Nisa Maulia Azahra (1301194136)
- Jerry Cahyo Setiawan (1301194022)
- Pramaishella Ardiani R.P (1301194105)

## Preprosesing Data

### Import Library

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from math import pi
from google.colab import files
from bokeh.io import output_notebook, curdoc
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, ColumnDataSource, CDSView, BooleanFilter, CategoricalColorMapper, Slider, Select, CheckboxButtonGroup
from bokeh.palettes import Category20, Spectral6
from bokeh.layouts import widgetbox, row, gridplot, column

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Dataset Initialization

Data yang digunakan pada tugas besar ini adalah Data Pergerakan Saham PT Media Nusantara Citra (MNC). Dataset tersebut memiliki atribut sebagai berikut:
- Date: Tanggal dari kumpulan data dikelompokkan.
- Open: Harga pembukaan/pertama pada transaksi yang dilakukan pada hari itu.
- High: Kisaran harga tertinggi dari pergerakan nilai saham pada hari itu.
- Low: Kisaran data terendah dari pergerakan nilai saham pada hari itu.
- Close: Harga penutupan/terakhir dari perdagangan saham berakhir pada hari itu.
- Volume: Jumlah saham yang berpindah tangan pada hari itu .

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/TubesVisualisasiData/MNCN.JK.csv")
# df.set_index("Date")
df.head()

,Date,Open,High,Low,Close,Volume
0,2015-07-06,1905,1910,1865,1875,4729600
1,2015-07-07,1895,1900,1835,1900,6288600
2,2015-07-08,1855,1890,1830,1870,6677300
3,2015-07-09,1850,1870,1830,1865,3503000
4,2015-07-10,1875,1890,1860,1860,4105800


## Interactive Visualization

Pada bagian ini ditampilkan grafik naik turunnya nilai saham pada PT Media Nusantara Citra dari tahun 2015 - 2021 menggunakan Candlestick dengan tambahan fitur pan, wheel zoom, box zoom, reset dan save.

In [ ]:
#MAIN DON'T TOUCH
df = pd.read_csv("/content/drive/MyDrive/TubesVisualisasiData/MNCN.JK.csv")
df["Date"] = pd.to_datetime(df["Date"])
df_red = df.loc[(df['Open'] >= df['Close'])]
df_green = df.loc[(df['Close'] >= df['Open'])]
w = 12*60*60*1000 # half day in ms

source = ColumnDataSource(df)
source_red = ColumnDataSource(df_red)
source_green = ColumnDataSource(df_green)

RED = Category20[7][6]
GREEN = Category20[5][4]
w = 12*60*60*1000

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

tooltips=[('open', '@Open'),
          ('close', '@Close'), 
          ('low', '@Low'), 
          ('high', '@High')]

p1 = figure(x_axis_type="datetime", 
            tools=TOOLS, 
            plot_width=1000, 
            plot_height=200, 
            title = "MNCN Candlestick with Volume")
p1.add_layout(Legend(click_policy="hide", orientation='vertical', spacing=20))
p1.sizing_mode = 'scale_both'

segmnts_green = p1.segment('Date', 'High', 'Date', 'Low', color=GREEN, source=source_green, legend_label='Up')
segmnts_red = p1.segment('Date', 'High', 'Date', 'Low', color=RED, source=source_red, legend_label='Down')
vbars_green = p1.vbar('Date', w, 'Open', 'Close', source=source_green, fill_color=GREEN, line_color=GREEN, legend_label='Up')
vbars_red = p1.vbar('Date', w, 'Close', 'Open', source=source_red, fill_color=RED, line_color=RED, legend_label='Down')

red_hover = HoverTool(renderers=[vbars_red], tooltips=tooltips)
p1.add_tools(red_hover)
green_hover = HoverTool(renderers=[vbars_green], tooltips=tooltips)
p1.add_tools(green_hover)

p2 = figure(x_axis_type="datetime", 
            tools=TOOLS, 
            plot_width=1500, 
            plot_height=200, 
            x_range=p1.x_range)
p2.vbar(df.Date, w, df.Volume, [0]*df.shape[0])

p = gridplot(([[p1],[p2]]), toolbar_location='above')

show(p)